# SUDOKU SOLVER

In questo notebook andremo a implementare due principali cose:
1. Una semplice gui per visualizzare il gioco del sudoku
2. Un solver che sfrutti metaeuristiche per risolvere il sudoku

Quali metaeuristiche andiamo a sviluppare?

La prima, una soluzione genetica, la seconda, un approccio del tipo simulated annealing.

Cofronteremo poi i risultati.

Ho aggiunto anche una versione che usa la programmazione lineare per risolvere i sudoku.

In [1]:
# i think we would use this so i import them to be sure
import numpy as np
import random
import math

Per i sudoku usiamo dei dati trovati su kaggle, così da semplificarci la vita. Lascio qui il link https://www.kaggle.com/datasets/bryanpark/sudoku?resource=download .

Ho aggiunto un limite al caricamento poiché per il nostro obiettivo, attualmente, non occorrono un milione di esempi.

In [2]:
limit = 1000
quizzes = np.zeros((limit, 81), np.int32)
solutions = np.zeros((limit, 81), np.int32)
for i, line in enumerate(open('sudoku.csv', 'r').read().splitlines()[1:limit]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s
quizzes = quizzes.reshape((-1, 9, 9))
solutions = solutions.reshape((-1, 9, 9))

Qui poi dedicheremo una parte a come visualizzare in maniera carina il sudoku. Penso userò Pygame.

In [3]:
import pygame 
def sudoku_grid_representation(sudoku, quiz, solution):
    pygame.init()
    
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    BLUE = (0, 0, 255)
    RED = (255, 0, 0)

    WINDOW_SIZE = 540
    GRID_SIZE = WINDOW_SIZE // 9
    LINE = WINDOW_SIZE // 3

    screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
    pygame.display.set_caption("Sudoku")

    def draw_grid():
        for row in range(9):
            for col in range(9):
                if quiz[row][col] != 0:
                    text = font.render(str(sudoku[row][col]), True, BLUE)
                    screen.blit(text, (col * GRID_SIZE + GRID_SIZE//3, row * GRID_SIZE + GRID_SIZE//3))
                elif sudoku[row][col] == solution[row, col]:
                    text = font.render(str(sudoku[row][col]), True, BLACK)
                    screen.blit(text, (col * GRID_SIZE + GRID_SIZE//3, row * GRID_SIZE + GRID_SIZE//3))
                else:
                    text = font.render(str(sudoku[row][col]), True, RED)
                    screen.blit(text, (col * GRID_SIZE + GRID_SIZE//3, row * GRID_SIZE + GRID_SIZE//3))
        for i in range (2):
            pygame.draw.line(screen, BLACK, (0, LINE * (i+1)),  (WINDOW_SIZE, LINE * (i+1)) )
            pygame.draw.line(screen, BLACK, (LINE * (i+1), 0), (LINE * (i+1), WINDOW_SIZE))

    font = pygame.font.Font(None, 36)
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()

        screen.fill(WHITE)
        draw_grid()
        
        pygame.display.flip()

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


Andiamo a vedere come costruire delle soluzioni.

Questa funzione trasforma un 9x9 in un 3x3x3x3 che rappresenta le celle.

In [4]:
def quiz_to_cells(quiz):
    cells = np.zeros((3, 3, 3, 3), np.int32)
    for i in range (3):
        for j in range (3):
            cells[i, j] = quiz[3*i:3*i+3, 3*j:3*j+3]
    return cells

Questo fa l'operazione inversa.

In [5]:
def cells_to_quiz(cells):
    quiz = np.zeros((9, 9), np.int32)
    for i in range (3):
        for j in range (3):
            quiz[3*i:3*i+3, 3*j:3*j+3] = cells[i, j]
    return quiz

Questa funzione completa (randomicamente) una cella.

In [6]:
def complete_cell(cell):
    cell_candidate = np.array(cell)
    cell_candidate = cell_candidate.reshape((9))
    missing_values = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    for i in range (9):
        if cell_candidate[i] != 0:
            missing_values.remove(cell_candidate[i])
    random.shuffle(missing_values)
    for i in range (9):
        if cell_candidate[i] == 0 :
            cell_candidate[i] = missing_values.pop()
    return cell_candidate.reshape((3, 3))

Questa completa (randomicamente) un quiz.

In [7]:
def compute_candidate(quiz):
    cells = quiz_to_cells(quiz)
    candidate_cells = np.zeros_like(cells)
    ga_solutions = [np.array(9)]
    for i, cell in enumerate(cells):
        for j, c in enumerate(cell):
            candidate_cells[i, j] = complete_cell(c)
    return candidate_cells

In [8]:
quiz = quizzes[0]
candidate = compute_candidate(quiz)

Scriviamo una funzione obiettivo.
La funzione obiettivo è nientemeno che: minimizzare la somma degli indicatori di valori unici mancanti lungo le righe e le colonne.

*Esempio:*

| | | | |valori unici mancanti |
|- |- |- |- |- |
|1 | 2| 1 | 2| **2**|
|3 | 4 | 4 | 3 | **2**|
|2 | 4 |  2| 3| **1**|
|1| 3 | 1 | 4| **1**|
|**1**| **1**|**1**|**1**| **<- valori unici mancanti** |

in questo caso abbiamo un totale di **10** valori unici mancanti.

La stessa funzione obiettivo può essere vista come minimizzare la somma degli indicatori di valori non unici lungo le righe e le colonne.

In [9]:
def same_element(vector):
    s = 0
    for i in range (1, 10):
        if i not in vector:
            s = s + 1
    return s

In [10]:
def object_function(candidate):
    rows = cells_to_quiz(candidate)
    cols = rows.T
    er = sum(np.array([same_element(r) for r in rows]))
    ec = sum(np.array([same_element(c) for c in cols]))
    return er + ec

### GA APPROACH

Partiamo con l'approccio genetico che è quello con cui siamo più ferrati.

Qui definiamo un crossover probabilistico. I crossover si eseguono sulle singole celle 3x3.

In [11]:
def p_crossover(a, b, p = 0.5):
    crossover = np.zeros_like(a)
    for i in range (3):
        for j in range (3):
            crossover[i, j] = a[i, j] if random.uniform(0, 1) > p else b[i, j]
    return crossover

Tipica Tournament Selection.

In [12]:
def tournament_selection(pop, k = 3):
    tournament = random.sample(pop, k)
    tournament.sort(key = object_function, reverse = False)
    return tournament[0]

Funzione d'appoggio per prendere degli indici a caso tra (0, 0) e (2, 2)

In [13]:
def random_index():
    k = random.randrange(0, 3)
    z = random.randrange(0, 3)
    return k, z

Mutazione per swap all'interno delle celle (chiaramente probabilistica).

In [14]:
def mutation(sol, quiz, mp = 0.9):
    map_quiz = quiz_to_cells(quiz)
    for i in range (3):
        for j in range (3):
            u = random.uniform(0, 1)
            if u < mp:
                
                k, z = random_index()                    
                while map_quiz[i, j, k, z] != 0:
                    k, z = random_index()
                w, v = random_index()
                while map_quiz[i, j, w, v] != 0:
                    w, v = random_index()
            
                t = sol[i, j, k, z]
                sol[i, j, k, z] = sol[i, j, w, v]
                sol[i, j, w, v] = t
    return sol          

Assembliamo il tutto.

In [15]:
def ga_sudoku_solver(pop, quiz, n_gen = 1000, eli = True, mp = 0.1, k = 3):
    pop.sort(key = object_function)
    sol = pop[0]

    while n_gen > 0 and object_function(sol) != 0:
        n_gen -= 1
        new_pop = []
        s = 0
        if eli == True:
            for i in range (k):
                new_pop.append(mutation(pop[i], quiz, mp))
            s = k
        while s < len(pop):
            parent_a = tournament_selection(pop, k)
            parent_b = tournament_selection(pop, k)
            son = p_crossover(parent_a, parent_b)
            son = mutation(son, quiz, mp)
            new_pop.append(son)
            s += 1
        pop = new_pop
        pop.sort(key = object_function)
        sol = pop[0]
    return sol       

Qua scriviamo una versione che permetta di visualizzare man mano ciò che accade.

In [16]:
def visual_ga_sudoku_solver(pop, quiz, solution, n_gen = 1000, eli = True, mp = 0.1, k = 3):
    pygame.init()

    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    BLUE = (0, 0, 255)
    RED = (255, 0, 0)

    WINDOW_SIZE = 540
    GRID_SIZE = WINDOW_SIZE // 9
    LINE = WINDOW_SIZE // 3

    screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
    pygame.display.set_caption("Sudoku")

    def draw_grid(sudoku, quiz, solution):
        for row in range(9):
            for col in range(9):
                if quiz[row][col] != 0:
                    text = font.render(str(sudoku[row][col]), True, BLUE)
                    screen.blit(text, (col * GRID_SIZE + GRID_SIZE//3, row * GRID_SIZE + GRID_SIZE//3))
                elif sudoku[row][col] == solution[row, col]:
                    text = font.render(str(sudoku[row][col]), True, BLACK)
                    screen.blit(text, (col * GRID_SIZE + GRID_SIZE//3, row * GRID_SIZE + GRID_SIZE//3))
                else:
                    text = font.render(str(sudoku[row][col]), True, RED)
                    screen.blit(text, (col * GRID_SIZE + GRID_SIZE//3, row * GRID_SIZE + GRID_SIZE//3))
        for i in range (2):
            pygame.draw.line(screen, BLACK, (0, LINE * (i+1)),  (WINDOW_SIZE, LINE * (i+1)) )
            pygame.draw.line(screen, BLACK, (LINE * (i+1), 0), (LINE * (i+1), WINDOW_SIZE))

    font = pygame.font.Font(None, 36)
    
    pop.sort(key = object_function)
    sol = pop[0]
    while n_gen > 0 and object_function(sol) != 0:
        n_gen -= 1
        new_pop = []
        s = 0
        if eli == True:
            for i in range (k):
                new_pop.append(mutation(pop[i], quiz, mp))
            s = k
        while s < len(pop):
            parent_a = tournament_selection(pop, k)
            parent_b = tournament_selection(pop, k)
            son = p_crossover(parent_a, parent_b)
            son = mutation(son, quiz, mp)
            new_pop.append(son)
            s += 1
        pop = new_pop
        pop.sort(key = object_function)
        sol = pop[0]
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()

        screen.fill(WHITE)
        draw_grid(cells_to_quiz(sol), quiz, solution)
        
        pygame.display.flip()
    pygame.quit()
    return sol

Prendiamo uno dei possibili quiz a caso tra quelli caricati e cominciamo a vedere i risultati.

In [17]:
r = random.randrange(0, limit)
quiz = quizzes[r]
solution = solutions[r]
POP = 100
population = [compute_candidate(quiz) for i in range (POP)]

In [18]:
#sol = visual_ga_sudoku_solver(population, quiz, solution, mp = 0.1)
#sudoku_grid_representation(cells_to_quiz(sol), quiz, solution)

In [19]:
#sudoku_grid_representation(quiz, quiz, solution)

In [20]:
#sol = ga_sudoku_solver(population, quizzes[r], n_gen=1000, mp = 0.1)
#print(object_function(sol))

### LINEAR PROGRAMMING APPROACH

Ora, mi è venuta questa idea e quindi ho detto "Massì, perché non provarci" e quindi eccomi qui, a scrivere un approccio di programmazione lineare per risolvere il sudoku.

In [21]:
import gurobipy as gp
from gurobipy import GRB

In [22]:
sudoku_model = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-26


In [23]:
sudoku_grid = sudoku_model.addMVar((9, 9, 9), vtype = GRB.BINARY)

In [24]:
sudoku_model.update()

Vincolo sull'unicità di ogni cella.

In [25]:
sudoku_model.addConstrs(gp.quicksum(sudoku_grid[l, i, j] for l in range (9)) == 1 for j in range (9) for i in range (9))
sudoku_model.update()

Vincolo sulle righe con tutti gli elementi diversi.

In [26]:
sudoku_model.addConstrs(gp.quicksum(sudoku_grid[l, i, j] for j in range (9)) == 1 for l in range (9) for i in range (9))
sudoku_model.update()

Vincolo sulle colonne con tutti gli elementi diversi.

In [27]:
sudoku_model.addConstrs(gp.quicksum(sudoku_grid[l, i, j] for i in range (9)) == 1 for j in range (9) for l in range (9))
sudoku_model.update()

Vincolo su tutte le celle con tutti elementi diversi.

In [28]:
sudoku_model.addConstrs(sum(gp.quicksum(sudoku_grid[l, 3*i:3*i+3, 3*j:3*j+3])) == 1 for l in range (9) for j in range (3) for i in range (3))
sudoku_model.update()

La funzione obiettivo non esiste poiché ogni soluzione va bene.

In [29]:
sudoku_model.setObjective(0)

In [30]:
sudoku_model.write("sudoku_model.mps")

Bisogna ora pensare ad un buon modo per costruire e decostruire una griglia di sudoku. Non è eccessivamente complesso.

Così de-costruiamo una griglia. Easy.

In [31]:
def vars_to_sudoku(sudoku_grid):
    sudoku = np.zeros((9, 9), np.int32)
    for i in range (9):
        for j in range (9):
            v = 0
            for l in range (9):
                if sudoku_grid[l, i, j].getAttr("X") == 1:
                    v = l + 1
            sudoku[i, j] = v
    return sudoku

In questo modo rendiamo un sudoku una griglia di lower bound.

In [32]:
def sudoku_to_vars(sudoku):
    vars_lower_bound = np.zeros((9, 9, 9), np.int32)
    for i in range (9):
        for j in range (9):
            v = sudoku[i, j]
            if v != 0:
                vars_lower_bound[v-1, i, j] = 1
    return vars_lower_bound

**ATTENZIONE**

Per poter eseguire tutta questa parte occorre avere una licenza gurobi poiché questo problema prevede 9^3 variabili decisionali e 9^3*9 (primi tre tipi di vincoli) + 81 (vincolo tra celle) + 9^3 vincoli (vincolo per il quiz esatto) e dunque la licenza gratuita non è sufficiente (7371 vincoli > 2000 licenza base).

Possiamo semplicemente aggiungerlo ed il gioco è fatto.

In [33]:
quiz = np.array([
    [4, 0, 0, 0, 0, 0, 3, 0, 7],
    [0, 0, 9, 0, 8, 0, 0, 2, 0],
    [0, 5, 0, 3, 0, 2, 0, 0, 6],
    [6, 9, 0, 7, 0, 0, 0, 5, 0],
    [0, 0, 3, 0, 0, 0, 7, 0, 8],
    [0, 0, 0, 0, 0, 5, 0, 0, 0],
    [0, 0, 0, 0, 0, 6, 0, 3, 0],
    [0, 0, 5, 0, 7, 9, 0, 0, 2],
    [2, 4, 0, 0, 0, 0, 8, 0, 0]
])
vars_lower = sudoku_to_vars(quiz)
sudoku_model.addConstrs(sudoku_grid[l, i, j] >= vars_lower[l, i, j] for l in range (9) for i in range (9) for j in range (9))
sudoku_model.update()
sudoku_model.optimize()


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1053 rows, 729 columns and 3645 nonzeros
Model fingerprint: 0x5f78fff4
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 984 rows and 666 columns
Presolve time: 0.02s
Presolved: 69 rows, 63 columns, 271 nonzeros
Variable types: 0 continuous, 63 integer (63 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, bes

In [34]:
sol = vars_to_sudoku(sudoku_grid)
sudoku_grid_representation(sol, quiz, solution)

error: display Surface quit

### SIMULATED ANNEALING APPROACH

Ora proviamo con questo approccio. E' completamente nuovo per cui sarà interessante.